### Previsão de venda de gasolina em galões no estado de Nova York usando a api Prophet - Facebook:
---
#### Sites para referencia:
---
##### Bar chart race: https://www.dexplo.org/bar_chart_race/data_preparation/
##### Forecast Prophet: https://juanitorduz.github.io/fb_prophet/
___

#### Processos de análise:
---
##### 1 - Configuração do notebook. 

##### 2 - Entrada de dados.

##### 3 - Transformação de dados.

##### 4 - Modelagem de series temporais no Prophet.

##### 5 - Previsão de vendas.

##### 6 - Análise gráfica.
---

#### Configuração do notebook:
---

In [ ]:
!pip install bar_chart_race
!pip install fbprophet

In [ ]:
# Pacotes de análise de dados:
import numpy as np
import pandas as pd
import seaborn as sns
import bar_chart_race as bcr
import matplotlib.pyplot as plt
%matplotlib inline

# Pacote de forecast Facebook Prophet:
import fbprophet
from fbprophet.plot import plot_plotly, plot_components_plotly
from sklearn.metrics import r2_score, mean_absolute_error

# Pacote para desenvolvimento em Python:
import tqdm as tqdm
import warnings
warnings.simplefilter("ignore")

#### Entrada de dados:
---

In [ ]:
# Entrada de dados:
df=pd.read_csv("../input/estimated-gasoline-sales/Estimated_Gasoline_Sales__Beginning_1995.csv")

#### Transformação de dados:
---

In [ ]:
# Dados de datas:
df = df.assign(Month=12,Day=31)
df.Month = df.Month.astype(str)
df.Day = df.Day.astype(str)
df.Year = df.Year.astype(str)
df = df.assign(ds=(df["Year"].map(str) + " " + df["Month"].map(str) + " " + df["Day"].map(str)),y= df["Estimated Sales, Thousands of Gallons"])
df.ds = pd.to_datetime(df.ds,format='%Y-%m-%d')

# Dados dos condados:
df = df[~df["County"].str.contains('Total')]
df = df[df["County"]!= "New York State"]
df = df[df["County"]!= "Alleghany"] #Allegany ajustar o nome da cidade

#### Análise explorátoria de dados:
---

In [ ]:
pd.DataFrame(df["Estimated Sales, Thousands of Gallons"].describe()).transpose().round(2)

In [ ]:
sns.relplot(kind='line',x='Year',y='Estimated Sales, Thousands of Gallons',data=df,col='County',col_wrap=4,facet_kws={'sharey': False, 'sharex': True})
plt.show()

In [ ]:
base = df.pivot(index='Year', columns='County', values='Estimated Sales, Thousands of Gallons')
bcr.bar_chart_race(df=base, 
                   n_bars=15, 
                   fixed_max=True,
                   perpendicular_bar_func='mean',
                   steps_per_period=8,
                   cmap='antique',
                   title='Estimated New York State Gasoline Sales',
                   shared_fontdict={'family': 'Helvetica', 'weight': 'bold',
                                    'color': 'rebeccapurple'})

#### Forecast - Prophet - Facebook:
---

In [ ]:
# Função de forecast - prophet - facebook:
def fun_train_model(df,cidade):
    # Parametros da função:
    cidades = df[cidade].unique()
    data4 = []
    
    # Forecast com Prophet:
    for i in cidades:
        # Preparação dos dados:
        data = df[df.County == i].sort_values(by = "ds")
        dataf = data[['ds','y']]

        # Modelagem de series temporais:
        m = fbprophet.Prophet(yearly_seasonality=True,weekly_seasonality=False,daily_seasonality=False,changepoint_prior_scale=1.5,seasonality_mode = 'additive')
        m.add_seasonality(name='yearly',period=1,fourier_order=10)
        m.fit(dataf)
        
        # Previsão:
        future = m.make_future_dataframe(periods=2,freq = "Y")
        forecast = m.predict(future)
        data2 = forecast[['ds','yhat','yhat_upper','yhat_lower']].round(2).head(len(dataf))
        print("County:" + i)
        #plot_plotly(m, forecast).show()
        
        # Base de resultados:
        data3 = data2.assign(y = dataf["y"].tolist(),County = i)
        
        # Métricas do treinamento:
        print("Métricas do treinamento:")
        print('r2: {}'.format(r2_score(y_true=data3['y'], y_pred=data3['yhat'].head(22)).round(2)))
        print('mae: {}'.format(mean_absolute_error(y_true=data3['y'], y_pred=data3['yhat'].head(22)).round(2)))
        print('rmse: {}'.format(np.sqrt(mean_absolute_error(y_true=data3['y'], y_pred=data3['yhat'].head(22)).round(2))),"\n")            
        
        # Criando base de dados com os resultados:
        data4.append(data3)
        
    # Criação do dataframe com os resultados
    data6 = pd.concat(data4,axis=0,ignore_index=True)
    return data6

#### Amostras de cidades:
---

In [ ]:
amostra = df[(df.County == "Orange") | (df.County == "Genesee")].sort_values(by = "ds")
amostra = amostra.reset_index(drop=True)

#### Forecast pelas cidades:
---

In [ ]:
base = fun_train_model(df,"County")

In [ ]:
base.head(2)

In [ ]:
base.tail(2)